In [30]:
# importing packages and libraries
from pandas import read_csv
import pandas as pd
import numpy as np
import pickle
import os
import random
import string
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

# Ignoe harmless warnings
import warnings
warnings.filterwarnings("ignore")

# Define the plot size default
from pylab import rcParams
rcParams['figure.figsize'] = (12, 5)

In [31]:
# new_data = 'new_data.csv'
new_data = 'valueflip_training_reduced_new.csv'
df_raw = pd.read_csv(new_data, index_col=0, header=0, parse_dates=True)
df_raw.index.freq = 'ms'

In [36]:
def normalise_timeseries(data):
    # Calculate the mean and standard deviation for each feature
    means = np.mean(data, axis=0)
    stds = np.std(data, axis=0)
    
    # Normalise each feature using standard deviation
    normalised_data = (data - means) / stds
#     return pd.DataFrame(normalised_data), means, stds
    return pd.DataFrame(normalised_data), means, stds


def denormalise_timeseries(data, means, stds):
    denormalised_data = (data * stds) + means
    return pd.DataFrame(denormalised_data)

def generate_random_name(length):
    # random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
    random_string = ''.join(random.choices(string.ascii_letters, k=length))
    # return "expert_models/New_expert_model_" + random_string + ".pkl"
    # return str("new_expert_model_" + random_string.lower() + ".pkl")
    return str("new_expert_model.pkl")




normalised_data, means, stds = normalise_timeseries(df_raw)
df = normalised_data.copy()

#  MAY BE YOU NEED TO REMOVE THE LINES BELOW
# Train Test Split
nobs = 3000
# train = df[:-nobs]
train = df.iloc[:-nobs]
test = df.iloc[-nobs:]
len(train), len(test)
# print(len(train))
# print(len(df))
train = df.copy()

In [40]:
# Perform order selection using auto-arima
model = VAR(train)
# Select the best order based on a specific criterion (e.g. AIC, BIC)
selected_order = model.select_order()
# Get the selected order (lag length)
order = selected_order.selected_orders['aic']
min_aic_index = order.copy()
results = model.fit(min_aic_index)

steps = 20
# z = results.forecast(y=train.values[-lag_order:], steps = 20)
z = results.forecast(y=train.values, steps=steps)

attr = list(pd.read_csv(new_data).columns.values)[1:]
df_forecast = pd.DataFrame(z, columns=attr)

# Save the expert model
expert_name = generate_random_name(8)

data = {'key': 'value'}

# Specify the subdirectory name
subdirectory = 'expert_models'

# Ensure the subdirectory exists, create it if necessary
os.makedirs(subdirectory, exist_ok=True)

# Construct the file path within the subdirectory
file_path = os.path.join(subdirectory, 'var_golden_models_reduced.pkl')

# Write data to a pickle file in the subdirectory
with open(file_path, 'wb') as f:
    pickle.dump(data, f)



# # with open('expert_models/new_expert_model_001.pkl', 'wb') as f:
# with open('expert_models/var_golden_models_reduced.pkl', 'wb') as f:
#     pickle.dump(results, f)

f.close()
print("Model saved!")

Model saved!
